# Introduction to Julia and Trixi, a numerical simulation framework for hyperbolic PDEs

[Julia](https://julialang.org) is a modern high-level programming language developed specifically with scientific computing in mind. [Trixi.jl](https://github.com/trixi-framework/Trixi.jl) is a numerical simulation framework for hyperbolic conservation laws written in Julia. A key objective for the framework is to be useful to both scientists and students. Therefore, next to having an extensible design with a fast implementation, Trixi is focused on being easy to use for new or inexperienced users, including the installation and postprocessing procedures.

This presentation is a live demonstration of Julia and Trixi. Firstly, we introduce Julia and demonstrate some of its design principles. This introduction is aimed at researchers in numerical analysis with previous programming experience. Next, we show how to use Trixi for setting up and running simulations, how to visualize the results, and how to extend Trixi with new functionality. We demonstrate how key design principles of Julia are used in Trixi and the Julia package ecosystem, e.g. to enable automatic differentiation through a complete simulation involving hyperbolic conservation laws.

The presentation is available as a Jupyter notebook at https://github.com/trixi-framework/talk-2021-Introduction_to_Julia_and_Trixi, including information how to set up everything. For more information about Trixi and how to use it, please visit [Trixi on GitHub](https://github.com/trixi-framework/Trixi.jl) or refer to the [official documentation](https://trixi-framework.github.io/Trixi.jl/stable/). 

This notebook was set up and tested with Julia v1.6.0 but may also work with other (newer) versions.

*Note:* If you change a variable in a later cell and then re-execute an earlier cell, the results might change unexpectedly. Thus if in doubt, re-run the entire notebook *in order*. The reason is that all cells in a Jupyter notebooks share a common variable space.

*Note:* This notebook is tested using Chromium. Most parts should also work for other browsers such as Firefox, but the videos used in the last demonstrations might not be displayed correctly.

This material is distributed by Hendrik Ranocha under the MIT license. It is inspired by and partially derived from the talks
- [Stefan Karpinski (2019), The unreasonable effectiveness of multiple dispatch](https://www.youtube.com/watch?v=kc9HwsxE1OY)
- [Robin Deits (2020), Intro to Julia Programming Language with Detroit Tech Watch](https://www.youtube.com/watch?v=qLO-yaUkLKE) 
- [Michael Schlottke-Lakemper (2021), Julia for adaptive high-order multi-physics simulations](https://github.com/trixi-framework/talk-2021-julia-adaptive-multi-physics-simulations)

In [ ]:
# Install all dependencies used in this talk
using Pkg
Pkg.activate(".")
Pkg.instantiate()

In [ ]:
ENV["COLUMNS"] = 100 # display width

In [ ]:
# Remove the margins when displayed as Jupyter notebook
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Display mp4 videos used in the demos
using Base64

function display_mp4(filename; relative_width=100, loop=true)
    loop_statement = loop ? " loop" : ""
    display("text/html", string(
    """
        <video autoplay controls width="$(relative_width)%"$(loop_statement)>
            <source src="data:video/x-m4v;base64,""",
            base64encode(open(read,filename)),
        """" type="video/mp4">
        </video>
    """))
end

<img src="media/titlepage.png" width="100%">

<!-- 
# Introduction to Julia and Trixi, a numerical simulation framework for hyperbolic PDEs

## Hendrik Ranocha 
 -->
 
- [ranocha.de](https://ranocha.de)
- Follow along at https://github.com/trixi-framework/talk-2021-Introduction_to_Julia_and_Trixi
- Launch MyBinder at https://tinyurl.com/fb2fw3hr

# Overview

- [Introduction to Julia](#Introduction-to-Julia)
  - [Julia is fast](#Julia-is-fast)
  - [What's hard to do in Julia?](#What's-hard-to-do-in-Julia?)
- [Solving hyperbolic PDEs with Trixi](#Solving-hyperbolic-PDEs-with-Trixi)
  - [Mixing an elixir: Creating a Trixi simulation from scratch](#Mixing-an-elixir:-Creating-a-Trixi-simulation-from-scratch)
  - [Advanced usage](#Advanced-usage)

# Introduction to Julia

- [julialang.org](https://julialang.org)
- Julia is a high-level language like Python/Matlab with the performance of a fast language like C/C++/Fortran
- Julia is designed for scientific computing...
  - N-dimensional arrays
  - Reproducibility
- ...and valuable for general programming
  - Growing ecosystem of packages
  - Rich type system
- Encourages good software development practices

## Julia at a glance

- First public release in 2012, version 1.0 released in 2018
- Free
  - Julia itself is MIT licensed
  - It bundles some GPLed libraries (which can be disabled if desired)
- Built-in JIT compiler transforms Julia code to native assembly at run time
  - Uses LLVM under the hood
- Garbage collected
- Dynamically typed
- Organized via multiple dispatch

## A brief tour of Julia

### The basics

In [ ]:
# Arithmetic
1 + 2

In [ ]:
# Strings
println("Hello world")

In [ ]:
# Arrays
x = [1, 2, 3]
sum(x)

### Unicode

In [ ]:
β = π / 4
tan(β) ≈ sin(β) / cos(β)

In [ ]:
using LinearAlgebra, Plots, LaTeXStrings
n = 1_000
λ = eigvals(randn(n, n))
scatter(real(λ), imag(λ), aspect_ratio=:equal, legend=nothing,
        xguide=L"\operatorname{Re} \lambda", yguide=L"\operatorname{Im} \lambda")

In [ ]:
# to free some memory on mybinder.org
λ = nothing
GC.gc()

### Functions

In [ ]:
function say_hello(to_whom)
    println("Hello ", to_whom)
end

In [ ]:
say_hello("world")

Functions are generic, so you can pass everything that works (duck typing)

In [ ]:
say_hello([1, 2, 3])

### Types

Everything in Julia has a type

In [ ]:
typeof(1)

In [ ]:
typeof(1.0)

In [ ]:
typeof(π)

In [ ]:
typeof([1, 2, 3])

You can create your own types easily

In [ ]:
struct Person
    name::String
end

alice = Person("Alice")

User-defined types are as efficient as anything built-in

In [ ]:
sizeof(Person) == sizeof(Ptr{String})

### Multiple dispatch

Julia does not use classes to to organize nouns (types) and verbs (functions). Instead, multiple dispatch is a central design decision. Thus, the compiler chooses an appropriate method of a given function based on the types of all arguments (not their values!).

For more information, see [Stefan Karpinski's talk at JuliaCon (2019)](https://www.youtube.com/watch?v=kc9HwsxE1OY).

In [ ]:
greet(x, y) = println(x, " greets ", y)

In [ ]:
alice = Person("Alice")
bob = Person("Bob")

greet(alice, bob)

Currently there is only one greet() function, and it will work on `x` and `y` of any type:

In [ ]:
greet(π, "Münster")

We can use abstract types to organize the behavior of related types:

In [ ]:
abstract type Animal end

struct Cat <: Animal
    name::String
end

We've already defined `greet(x, y)` for any `x` and `y`, but we can add another definition for a more specific set of input types.

We can be as specific or as general as we like with the argument types:

In [ ]:
greet(x::Person, y::Animal) = println(x, " pats ", y)

In [ ]:
greet(x::Cat, y) = println(x, " meows at ", y)

Julia will always pick the *most specific* method that matches the provided function arguments.

In [ ]:
fluffy = Cat("Fluffy")

greet(alice, fluffy)

In [ ]:
greet(fluffy, alice)

In [ ]:
struct Dog <: Animal
    name::String
end

greet(x::Dog, y) = println(x, " barks at ", y)

greet(x::Dog, y::Person) = println("$x licks $y's face")

greet(x::Dog, y::Dog) = println("$x sniffs $y's butt")

In [ ]:
fido = Dog("Fido")
rex = Dog("Rex")

greet(alice, fido)

In [ ]:
greet(fido, fluffy)

In [ ]:
greet(fido, bob)

In [ ]:
greet(fido, rex)

If you want to know which `greet` method will be called for a given set of arguments, you can use `@which` to check:

In [ ]:
@which greet(alice, fido)

You can list all of the methods of a given function with `methods`:

In [ ]:
methods(greet)

## Modules

Modules in Julia are used to organize code into namespaces.

In [ ]:
module MyUsefulModule

export hello

hello()   = println("Hello world")
goodbye() = println("Goodbye world")

end

MyUsefulModule.hello()

The `using` command brings any `export`ed symbols from a module into the current namespace:

In [ ]:
using .MyUsefulModule
hello()

## Using packages

Julia has a built-in package manager called `Pkg`. It handles installing packages and managing all your package environments. 

A package *environment* represents a single set of installed packages. Let's activate the environment for this talk:

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

(this is similar to `source venv/bin/activate` in a Python virtual environment)

We can install a package in our current environment. This will only affect that environment, so we can safely do this without breaking any other Julia projects we might be working on:

In [ ]:
Pkg.add("BenchmarkTools")

The `Project.toml` file gives a concise description of the packages we've added to this environment:

In [ ]:
run(`cat Project.toml`)

The package manager also generates a complete manifest of every package that is installed, including all the transitive dependencies and their versions. You can use this to reproduce a given package environment exactly:

In [ ]:
run(`head Manifest.toml`)

# Julia is fast

* I claimed at the beginning of this talk that Julia has performance on par with C. Let's prove it!
* To show this, I'll implement the basic `sum` function in Julia, C, and Python so we can compare them:

Let's start with Julia:

In [ ]:
"""
    my_sum(x)

Naive implementation of sum. Works for any iterable `x` with any element type.
"""
function my_sum(x)
    result = zero(eltype(x))
    for element in x
        result += element
    end
    return result
end

And let's create some data to test with:

In [ ]:
data = randn(Float64, 10^7);

To measure the performance of `my_sum`, we'll use the [BenchmarkTools](https://github.com/JuliaCI/BenchmarkTools.jl) package. 

In [ ]:
using BenchmarkTools

In [ ]:
@benchmark my_sum($data)

In this case, we only care about the minimum time. The `@btime` macro is a shorthand to print just that minimum time:

In [ ]:
@btime my_sum($data)

Let's compare this with C. It's easy to call functions from C shared libraries in Julia:

In [ ]:
"""
Call the `strcmp` function from `libc.so.6`
"""
function c_compare(x::String, y::String)
    # We have to tell the compiler that this C function returns an `int` and 
    # expects two `char *` inputs. The `Cint` and `Cstring` types are convenient
    # shorthands for those:
    ccall((:strcmp, "libc.so.6"), Cint, (Cstring, Cstring), x, y)
end

In [ ]:
c_compare("hello", "hello")

Calling C functions has very little overhead:

In [ ]:
@btime c_compare($("hello"), $("hello"))

Let's create a C implementation of `my_sum`. We can do that without leaving Julia by piping some code directly to GCC:

In [ ]:
C_code = """

#include <stddef.h>  // For `size_t`

// Note: our Julia code works for any type, but the C implementation 
// is only for `double`.

double c_sum(size_t n, double* x) {
    double s = 0.0;
    size_t i;
    for (i = 0; i < n; ++i) {
        s += x[i];
    }
    return s;
}

""";

Now let's generate a name for our shared library:

In [ ]:
# dlext gives the correct file extension for a shared library on this platform
using Libdl: dlext
Clib = tempname() * "." * dlext

To send the code to GCC, we can use `open()` on a command to write directly to the `stdin` of that command as if it were any other file- or buffer-like object:

In [ ]:
open(`gcc -fPIC -O3 -march=native -mtune=native -xc -shared -o $Clib -`, "w") do cmd
    print(cmd, C_code) 
end

Now we can define a Julia function that calls the C function we just compiled:

In [ ]:
# The return type and argument types must match the signature we declared above:
# 
#   double c_sum(size_t n, double* x) 
# 
c_sum(x::Array{Float64}) = @ccall Clib.c_sum(length(x)::Csize_t, x::Ptr{Cdouble})::Cdouble

Now let's measure the performance of the pure C function:

In [ ]:
@btime c_sum($data)

Let's plot the result using the [Plots](https://github.com/JuliaPlots/Plots.jl) package.

In [ ]:
using Plots

In [ ]:
results = Dict(
    "my_sum (Julia)" => 9.2,
    "c_sum (C)" => 9.1
)

xlabel="function"; ylabel="time (ms, shorter is better)"; legend=nothing;
bar(results; xlabel, ylabel, legend)

Our naive Julia code is just as fast as our naive C code! 

Is that as fast as we can go? What about Julia's built-in `sum()` function:

In [ ]:
@btime sum($data)

In [ ]:
results["sum (Julia)"] = 3.4

bar(results; xlabel, ylabel, legend)

What's going on? Is the `sum()` function using some built-in behavior we don't have access to?

Nope - we can achieve that result easily with a few modifications:

In [ ]:
function my_fast_sum(x)
    result = zero(eltype(x))

    # `@simd` enables additional vector operations by indicating that it is OK to potentially
    # evaluate the loop out-of-order. 
    @simd for element in x
        result += element
    end
    result
end

In [ ]:
@btime my_fast_sum($data)

In [ ]:
results["my_fast_sum (Julia)"] = 3.4

bar(results; xlabel, ylabel, legend)

With some pretty simple changes, we were able to create a pure-Julia function which is twice as fast as our naive C function while still being clear and completely generic:

In [ ]:
my_fast_sum([1, 2.5, π])

Just for reference, let's compare with Python. It's easy to call Python code from Julia too - we just need the [PyCall](https://github.com/JuliaPy/PyCall.jl) package:

In [ ]:
using PyCall

In [ ]:
py_math = pyimport("math")
py_math.sin.([0, π/2, π])

Just as we did with C, we can quickly define a Python sum function without leaving Julia:

In [ ]:
# The PyCall package lets us define python functions directly from Julia:

py"""
def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

# mysum_py is a reference to the Python mysum function
py_sum = py"""mysum"""o

Let's make sure we're getting similar answers everywhere:

In [ ]:
py_sum(data) ≈ c_sum(data) ≈ sum(data) ≈ my_sum(data) ≈ my_fast_sum(data)

In [ ]:
@btime py_sum($data)

In [ ]:
results["py_sum (Python)"] = 1.4e3

bar(results; xlabel, ylabel, legend)

### What about Numpy or Cython?

* Of course, there are faster ways to sum a vector of `double`s in Python than a `for` loop. 
* `numpy.sum()` is just as fast as Julia's `sum()` for large vectors...
* ...but there are some caveats:
  * NumPy is only efficient for a pre-determined set of numeric types. 
  * NumPy cannot be extended without switching into an entirely different programming language, build system, and code environment. 
  * So, if `numpy.sum()` happens to cover the cases you actually need, then go for it!
  * But if you want to be able to write efficient code that does not happen to cover the specific set of functions and types in NumPy, then you need Julia. 

In [ ]:
struct Point{T}
    x::T
    y::T
end

function Base.zero(::Type{Point{T}}) where {T} 
    Point{T}(zero(T), zero(T))
end
    
Base.:+(p1::Point, p2::Point) = Point(p1.x + p2.x, p1.y + p2.y)

n = 10^7; points = Point.(randn(n), randn(n));

In [ ]:
@btime my_fast_sum($points)

In [ ]:
@code_native debuginfo=:none my_fast_sum(points)

# Bonus features of Julia

## Anything can be a value

Julia has no special rules about what can or cannot be assigned to a variable or passed to a function. 

### Functions are values

A Julia function is a value like any other, so passing functions around and implementing higher-order functions is trivial:

In [ ]:
"""
map_reduce: apply `operator` to each element in `array` and reduce pairwise via `reduction`
"""
function map_reduce(operator, reduction, array, initial_value)
    result = initial_value
    for item in array
        result = reduction(result, operator(item))
    end
    result
end

In [ ]:
map_reduce(inv, +, [1, 2, 3], 0)

We can define `sum` in terms of `map_reduce`:

In [ ]:
fancy_sum(x) = map_reduce(identity, +, x, zero(eltype(x)))

The performance is just as good as our hand-written `sum` loop:

In [ ]:
@btime fancy_sum($data)

To get the same optimized performance, we'd need to apply the same `@simd` annotations.  

Functions can be inlined, even into standard library code. The compiler heuristics are often good and you can nudge it using `@inline` if necessary.

In [ ]:
my_identity(x) = x
my_plus(x, y) = x + y
@btime mapreduce($my_identity, $my_plus, $data)

In [ ]:
# to free some memory on mybinder.org
data = nothing
points = nothing
GC.gc()

### Types are values

Types can also be passed around as values and bound to variables with no special rules. This makes implementing factories or constructors easy:

In [ ]:
function empty_matrix(T::Type, rows::Integer, cols::Integer)
    zeros(T, rows, cols)
end

In [ ]:
empty_matrix(Int, 3, 3)

In [ ]:
empty_matrix(Point{Float64}, 3, 3)

### Expressions are values

Even the expressions that representing Julia code are represented as values in Julia. You can create an expression with the `:()` operator, and you can inspect it just like any other object. 

In [ ]:
expr = :(1 + 2)

An expression has a `head` indicating what type of expression it is and zero or more `args`:

In [ ]:
expr.head

In [ ]:
expr.args

## Metaprogramming

Since expressions are just values, we can easily write functions to manipulate them:

In [ ]:
switch_to_subtraction!(x::Any) = nothing

"""
Change all `+` function calls to `-` function calls. 

<sarcasm>
Great for fixing sign errors in your code!
</sarcasm>
"""
function switch_to_subtraction!(ex::Expr)
    if ex.head == :call && ex.args[1] == :(+)
        ex.args[1] = :(-)
    end
    for i in 2:length(ex.args)
        switch_to_subtraction!(ex.args[i])
    end
end

In [ ]:
expr = :((1 + 2) * (3 + 4) * sqrt(2))

In [ ]:
switch_to_subtraction!(expr)

expr

### Macros

A macro is written just like a normal Julia function. The difference is that a macro operates on the *expression* itself, not on its value:

`@show` : print out the *name* of a variable and its value. Great for quick debugging:

In [ ]:
x = 5
@show x

`@time` measure the elapsed time of an expression and return the result of that expression:

In [ ]:
@time sqrt(big(π))

We have seen its siblings `@btime` and `@benchmark` above.

`@showprogress`: Time each iteration of a loop and estimate how much longer it will take to finish:

In [ ]:
using ProgressMeter: @showprogress

In [ ]:
@showprogress for i in 1:100
    sum(randn(10^7))
end

# What's hard to do in Julia?

- What is the compiler team working on making better?
- What are some subtle problems that the Julia team would like to fix?

[![What's bad about Julia talk](img/bad_about_julia.png)](https://www.youtube.com/watch?v=TPuJsgyu87U)

## Compiler latency

- The JIT compiler runs each time it sees a function being called with a new input type
- That makes the first call slow, since you have to wait for the JIT
  - This makes Julia awkward for things like shell scripts or AWS lambda
- The compiler is essentially serial

## Static compilation

* To avoid the JIT lag, you can compile a Julia package to a standalone executable using [PackageCompiler.jl](https://github.com/JuliaLang/PackageCompiler.jl), but:
    * This workflow is pretty new, and you may run into interesting bugs
    * The resulting libraries tend to be quite large

## Static analysis

* There are some linting tools for Julia (like the `vscode-julia` extension for Visual Studio Code), but they are not as mature as languages like Python, C, Java, etc.
* Static analysis of Julia is harder, since the language itself is dynamically typed
  * To be fair, static analysis of C++ is [undecidable](https://blog.reverberate.org/2013/08/parsing-c-is-literally-undecidable.html) but we still have tools that do a pretty good job most of the time

## HPC for simulation science (in contrast to data science)

- The ecosystem for traditional HPC frameworks needs to be advanced
- Binaries (depending on MPI) must still be configured manually
- Some momemtum comes from the [CliMA project at CalTech](https://clima.caltech.edu/)

## Embedded computing

- It can be hard to run Julia on memory-limited systems, since you need the compiler living alongside your code
- Static compilation can help, but this isn't a well-developed workflow yet

# Useful Julia Tools

## Julia VSCode

https://github.com/julia-vscode/julia-vscode

* Code highlighting, snippets, linting, and completions
* Integrated plot and table viewers
* General extension support via the VSCode language server

<div align="center">
  <img src="https://github.com/julia-vscode/julia-vscode.github.io/raw/243947a5d00b47ff65b9c133da287697f84aeada/img/newscreen5.png" width="70%"/>
</div>

## [Revise](https://github.com/timholy/Revise.jl)

- Reduce overhead while developing
- Partial replacement of `make`

<div align="center">
  <img src="https://github.com/timholy/Revise.jl/raw/master/images/revise-wordmark.png" width="60%">
</div>

## [DifferentialEquations/OrdinaryDiffEq](https://github.com/SciML/DifferentialEquations.jl)

* { ordinary | delay | stochastic | * } differential equations
* Automatic differentiation and sparsity detection
* Sensitivity analysis and parameter estimation
* Access to pure-Julia solvers and existing C and Fortran solvers

<div align="center">
  <img src="https://github.com/SciML/DifferentialEquations.jl/raw/master/assets/DifferentialEquations_Example.png" width="50%"/>
</div>


## [Flux](https://github.com/FluxML/Flux.jl)

- Flexible library for machine learning built entirely in Julia
- Feed-forward and recurrent neural nets
- Gradients via automatic differentiation
- GPU support via CuArrays.jl

```julia
m = Chain(
  Dense(784, 32, σ),
  Dense(32, 10), softmax
)

loss(x, y) = Flux.mse(m(x), y)
ps = Flux.params(m)

for i in 1:num_training_iters
    Flux.train!(loss, ps, data, opt)
end
```

## [DataFrames](https://github.com/JuliaData/DataFrames.jl)

- In-memory tabular data
- Joining, indexing, grouping, and split-apply-combine

```julia
julia> using DataFrames

julia> df = DataFrame(A = 1:4, B = ["M", "F", "F", "M"])
4×2 DataFrame
│ Row │ A     │ B      │
│     │ Int64 │ String │
├─────┼───────┼────────┤
│ 1   │ 1     │ M      │
│ 2   │ 2     │ F      │
│ 3   │ 3     │ F      │
│ 4   │ 4     │ M      │
```

## [JuMP](https://github.com/jump-dev/JuMP.jl)

- Continuous and discrete optimization
- Support for a wide variety of free and commercial solvers
- Efficient high-level language for mathematical programming

<div align="center">
  <img src="https://raw.githubusercontent.com/jump-dev/JuMP.jl/master/docs/src/assets/jump-logo-with-text.svg" width="60%"/>
</div>



## [Cassette](https://github.com/JuliaLabs/Cassette.jl)

https://www.youtube.com/watch?v=_E2zEzNEy-8

"Overdub" Julia code: 

- Hook into the compiler to dynamically modify the behavior of existing functions in a given *context*
- Building block for debuggers, automatic differentation, mocking/testing frameworks, and more.

<div align="center">
  <img src="https://github.com/JuliaLabs/Cassette.jl/raw/master/docs/img/cassette-logo.png" width="60%"/>
</div>

## Automatic differentiation

- Forward/reverse mode AD and more
  - [ForwardDiff](https://github.com/JuliaDiff/ForwardDiff.jl)
  - [ReverseDiff](https://github.com/JuliaDiff/ReverseDiff.jl)
  - [Zygote](https://github.com/FluxML/Zygote.jl)
  - More to come
- Common tools in [ChainRules](https://github.com/JuliaDiff/ChainRules.jl)

<div align="center">
  <img src="https://avatars.githubusercontent.com/u/7750915?s=200&v=4" width="20%">
</div>

## [LoopVectorization](https://github.com/JuliaSIMD/LoopVectorization.jl), [CheapThreads](https://github.com/JuliaSIMD/CheapThreads.jl)

- Advanced SIMD programming in Julia
- Able to beat MKL with pure Julia code

<div align="center">
  <img src="https://avatars.githubusercontent.com/u/80543003?s=200&v=4" width="20%">
</div>

## [JuliaGPU](https://juliagpu.org/)

- Compile Julia on GPUs
- High-level interface used for ML, data science, differential equations, ...

<div align="center">
  <img src="https://github.com/JuliaGPU/juliagpu.org/raw/master/_assets/logo_crop.png" width="20%">
</div>

# Solving hyperbolic PDEs with Trixi

- First code in January 2020
- Version `v0.1` on GitHub [in August 2020](https://discourse.julialang.org/t/ann-trixi-jl-a-tree-based-numerical-simulation-framework-for-hyperbolic-pdes/45886)
- Version `v0.3` published [in November 2020](https://discourse.julialang.org/t/ann-trixi-jl-v0-3-sciml-integration-and-a-new-modular-approach-for-easy-extension/50419)
- [12 active contributors](https://github.com/trixi-framework/Trixi.jl/graphs/contributors)

<div align="center">
  <img src="media/Authors.png" width="80%">
</div>

## Quickstart

Let's have a look at a simple 2D example.

`trixi_include(...)` is a function that loads a Julia file and executes its contents. We call such files that contain a valid Trixi setup **elixirs**. `default_example()` is part of the Trixi package and returns the path to an example elixir for running a 2D linear advection simulation.

**Please note that during the first invocation this may take a minute or two, since Julia has to compile all functions at first usage.**

In [ ]:
using Trixi
trixi_include(default_example())

The results of a Trixi simulation can easily be visualized with the [Plots](https://github.com/JuliaPlots/Plots.jl) package. Simply load `Plots` and call the `plot(...)` command with the argument `sol`. By convention, in all our elixirs `sol` (short for *solution*) is the variable name that contains the result of a simulation.

In [ ]:
using Plots
plot(sol)

# Mixing an elixir: Creating a Trixi simulation from scratch

Let's have a look at what an elixir looks like. Generally speaking, the following components are required:
- an **equation** object that contains all physics-specific data and functionality
- a **solver** that represents the numerical method and its algorithms
- a **mesh** that holds the grid data
- a **semidiscretization** that encapsulates equation, solver, and mesh, together with initial and boundary conditions

The semidiscretization object is then used to create an `ODEProblem` that can be solved with one of the solvers for ordinary differential equations (ODEs) from the [OrdinaryDiffEq](https://github.com/SciML/OrdinaryDiffEq.jl) package.

<div align="center">
  <img src="https://github.com/trixi-framework/talk-2021-Introduction_to_Julia_and_Trixi/raw/main/media/components.png" width="80%" />
</div>

In the following are the contents of a *minimum* elixir that produces the same result as the `trixi_include(...)` command above (the elixir can be found in [examples/elixir_advection_simple.jl](examples/elixir_advection_simple.jl)):

In [ ]:
using OrdinaryDiffEq, Trixi

In [ ]:
advectionvelocity = (1.0, 1.0)
equations = LinearScalarAdvectionEquation2D(advectionvelocity)

In [ ]:
solver = DGSEM(polydeg=3, surface_flux=flux_lax_friedrichs)

In [ ]:
# Create a uniformely refined mesh with periodic boundaries
coordinates_min = (-1.0, -1.0) # minimum coordinates
coordinates_max = ( 1.0,  1.0) # maximum coordinates
mesh_static = TreeMesh(coordinates_min, coordinates_max,
                       initial_refinement_level=4, n_cells_max=10^5)

In [ ]:
# Create semidiscretization with all spatial discretization-related components
semi = SemidiscretizationHyperbolic(mesh_static, equations,
                                    initial_condition_convergence_test,
                                    solver)

# Create ODE problem from semidiscretization with time span from 0.0 to 1.0
ode = semidiscretize(semi, (0.0, 1.0));

# Evolve ODE problem in time using OrdinaryDiffEq's `solve` method
@time sol = solve(ode, BS3(), save_everystep=false);

And that's it. In this minimum example, there is no output to the terminal/notebook, thus only the lack of errors tells us that everything went smoothly. However, as before, we can visualize the solution by plotting it with the `plot` function of the `Plots` package:

In [ ]:
using Plots
plot(sol)

As you can see from the snippet above, the entirety of a simulation setup is defined in pure Julia: There are no "special" parameter files, and changing a simulation setup means to modify its elixir. For example, we can change the advection speeds by passing a different velocity vector to `LinearScalarAdvectionEquation2D`,

```julia
# Create equations with different advection velocity
advectionvelocity = (1.0, 0.1)
equations = LinearScalarAdvectionEquation2D(advectionvelocity);
```

and re-run the simulation by re-creating the semidiscretization and the `ODEProblem`, and then calling the `solve(...)` function again.

## Using a different initial condition

So far we have only used functionality that is provided by Trixi. With its modular architecture, however, it is easy to extend Trixi with your own features. Let's define a new initial condition with a cosine-shaped pulse at its center:

In [ ]:
using LinearAlgebra # for `norm`

function cosine_pulse(x, t, equations::LinearScalarAdvectionEquation2D)
  halfwidth = 0.5
  radius = norm(x)

  if radius > halfwidth
    u = 0.0
  else
    u = 0.1 + 0.1 * cospi(radius / halfwidth)
  end

  return SVector(u)
end

Now we can use this function for the initial condition parameter in the semidiscretization, re-run the simulation, and plot the results:

In [ ]:
# Recreate semidiscretization with the new initial condition
semi = SemidiscretizationHyperbolic(mesh_static, equations,
                                    cosine_pulse, # <-- here is the new initialization function
                                    solver)
# Create and solve ODE problem
ode = semidiscretize(semi, (0.0, 1.7));
sol = solve(ode, BS3(), saveat=range(ode.tspan..., length=6));

using Printf
plot(map(i -> plot(sol.u[i], semi, title=@sprintf("\$ t = %.2f \$", sol.t[i])), 
        eachindex(sol.t))..., size=(1200, 550))

## Adaptive mesh refinement

In the previous examples, we have always used a static, uniform mesh. This means we did not fully exploit one of Trixi's fundamental building blocks: the underlying hierarchical Cartesian mesh. With its ability to locally refine the mesh in a solution-adaptive way, it can be used to greatly speed up simulations with no or minimal loss in overall accuracy. The following image series of the grid for an expanding blast wave simulation ([elixir](https://github.com/trixi-framework/Trixi.jl/blob/v0.3.7/examples/2d/elixir_euler_blast_wave_amr.jl) available with Trixi) at $t = [0.2, 0.6, 1.0]$ (left to right) illustrates how the mesh resolution is increased only where needed.

<div align="center">
  <img src="https://github.com/trixi-framework/talk-2021-Introduction_to_Julia_and_Trixi/raw/main/media/callbacks_AMR.png" width="80%" />
</div>

With Trixi, you can statically refine the mesh upfront, adaptively refine it during a simulation, or use a combination of both. Here, we will focus on how to do adaptive mesh refinement (AMR), which is implemented in Trixi as a **step callback**. A callback is an algorithmic entity that is registered with the ODE solver and then *called* in regular intervals to perform certain tasks: *step* callbacks are run after each time step, while *stage* callbacks are run after each stage of the ODE solver.

Using callbacks instead of hard-coding all features in the main loop has the advantage that it allows to extend Trixi with new functionality without having to modify its code directly. Beyond AMR, in Trixi we make use of callbacks also for tasks such as solution analysis, file I/O, in-situ visualization, or time step size calculation.

Building upon the previous example, we will now introduce the `AMRCallback` to adaptively refine the mesh around the cosine pulse every 5 steps. The callback also requires the definition of an *AMR controller*, i.e., an object that tells the AMR algorithms which cells to refine/coarsen:

In [ ]:
# Use a simple AMR controller that to refine cells between level 4 and level 6,
# based on the maximum value of the solution in an element
amr_controller = ControllerThreeLevel(semi,
                                      IndicatorMax(semi, variable=first),
                                      base_level=4,
                                      med_level=5, med_threshold=0.05,
                                      max_level=6, max_threshold=0.15);

# Create the AMR callback that will be called every 5 time steps
amr_callback = AMRCallback(semi, amr_controller, interval=5)

The `AMRCallback` can now be passed to the ODE solver and will be invoked (i.e., performs adaptive mesh refinement) every 5 time steps. Note that we also create a new mesh with a lower initial refinement, since by default the `AMRCallback` will iteratively refine the initial mesh based on the AMR controller until it does not change anymore.

In [ ]:
# Create new mesh and semidiscretization with lower initial refinement level
mesh_amr = TreeMesh(coordinates_min, coordinates_max,
                    initial_refinement_level=2,
                    n_cells_max=30_000)
semi = SemidiscretizationHyperbolic(mesh_amr, equations,
                                    cosine_pulse,
                                    solver)

# Create and solve ODE problem, using the previously constructed AMR callback
ode = semidiscretize(semi, (0.0, 1.7));
sol = solve(ode, BS3(), save_everystep=false, callback=amr_callback);

# Plot result
pd = PlotData2D(sol)
plot(pd, seriescolor=:heat)
plot!(getmesh(pd))

Here we also changed the color scheme and added the mesh lines to the plot to demonstrate how the mesh is locally refined around the cosine pulse. To find out more about how to visualize solutions with the `Plots` package, please refer to the [Trixi documentation](https://trixi-framework.github.io/Trixi.jl/stable/visualization/#Plots.jl).

# Advanced usage

In the following, we will introduce some of the more advanced features in Trixi (advanced in the sense that they go beyond basic functionality, not that they are particularly complicated to use).

## Analyzing the solution

Oftentimes it is desireable to analyze a running simulation quantitatively by calculating integral quantities on the fly, which can be achieved by creating an `AnalysisCallback`. By default, it computes the $L^2$ and $L^\infty$ errors and prints it to the terminal (we have seen an example of this in the [introduction]((#Solving-hyperbolic-PDEs-with-Trixi)) section above). This can be extended to other built-in or user-provided integral quantities, such as the entropy time derivative $\partial S/\partial u \cdot \partial u/\partial t$, the conservation error, or the total kinetic energy.

In the following, we will create and `AnalysisCallback` that performs a solution analysis every 20 time steps, additionally computes the conservation error, and saves everything to a file (in addition to showing it on the terminal):

In [ ]:
analysis_callback = AnalysisCallback(semi, interval=20,
                                     extra_analysis_errors=(:conservation_error,));

Since the $L^2$ and $L^\infty$ errors are computed with respect to the (time-resolved) initial condition function, we will again use the  fully periodic`initial_condition_convergence_test` to initialize the solution. We will also use a static mesh again.

In [ ]:
# Create semidiscretization with a fully periodic initial condition
semi = SemidiscretizationHyperbolic(mesh_static, equations, initial_condition_convergence_test, solver)

# Create and solve ODE problem
ode = semidiscretize(semi, (0.0, 1.0));
sol = solve(ode, BS3(), save_everystep=false, callback=analysis_callback);

## Running convergence tests

When developing new numerical methods, a common part of the workflow is the check if the implemented scheme still exhibits the expected order of convergence (EOC) for mesh refinement. Trixi has a helper function `convergence_test(...)`, which re-runs a given elixir multiple times, each time increasing the mesh resolution by one. For this, we will again use the default example elixir provided by Trixi.

*Note:* Since changing the mesh resolution also affects the allowed time step through the CFL condition, `convergence_test(...)` requires the elixir to use a [StepsizeCallback](https://trixi-framework.github.io/Trixi.jl/stable/reference-trixi/#Trixi.StepsizeCallback) to automatically compute the explicit time step.

In [ ]:
# Run convergence test with default example and 3 different mesh refinement levels
convergence_test(default_example(), 3);

## Visualizing the spectrum

Trixi contains a method `linear_structure` that wraps the right-hand side operator of a semidiscretization as a linear operator. This can then be used to plot the spectrum for, e.g., the discretization of the scalar advection equation:

In [ ]:
# Include an elixir to quickly obtain an appropriate semi discretization
trixi_include(joinpath("examples", "elixir_advection_simple.jl"), initial_refinement_level=2)

# Get linear operator
A, b = linear_structure(semi)

# Compute eigenvalues
using LinearAlgebra
λ = eigvals(Matrix(A))

# Plot eigenvalues in complex plane
plot(real(λ), imag(λ), seriestype=:scatter, legend=nothing)

Similarly, to analyse the spectrum of non-linear operators we can use `jacobian_ad_forward`, which uses sforward mode automatic differentiation to compute the Jacobian `J` of the operator. Here we use the elixir [examples/elixir_euler_simple.jl](examples/elixir_euler_simple.jl), which is a simplified version of an elixir for the compressible Euler equations that also comes packaged with Trixi:

In [ ]:
# Include an elixir to quickly obtain an appropriate semidiscretization
trixi_include(joinpath("examples", "elixir_euler_simple.jl"))

# Determine Jacobian
J = jacobian_ad_forward(semi)

# Compute eigenvalues
λ = eigvals(J)

# Plot eigenvalues in complex plane
plot(real(λ), imag(λ), seriestype=:scatter, legend=nothing)

## Differentiating through a complete simulation

More details can be found in the [documentation of Trixi](https://trixi-framework.github.io/Trixi.jl).

In [ ]:
using ForwardDiff

function energy_at_final_time(k) # k is the wave number of the initial condition
   equations = LinearScalarAdvectionEquation2D(1.0, -0.3)
   mesh = TreeMesh((-1.0, -1.0), (1.0, 1.0), initial_refinement_level=3, n_cells_max=10^4)
   solver = DGSEM(3, flux_lax_friedrichs)
   initial_condition = (x, t, equation) -> begin
       x_trans = Trixi.x_trans_periodic_2d(x - equation.advectionvelocity * t)
       return SVector(sinpi(k * sum(x_trans)))
   end
   semi = SemidiscretizationHyperbolic(mesh, equations, initial_condition, solver,
                                       uEltype=typeof(k))
   ode = semidiscretize(semi, (0.0, 1.0))
   sol = solve(ode, BS3(), save_everystep=false)
   Trixi.integrate(energy_total, sol.u[end], semi)
end

k_values = range(0.9, 1.1, length=101)

In [ ]:
energy_values = energy_at_final_time.(k_values)
plot(k_values, energy_values .- maximum(energy_values), 
     xguide=L"k", label="Relative energy")

You should see a plot of a curve that resembles a parabola with local maximum around `k = 1.0`. Why's that? Well, the domain is fixed but the wave number changes. Thus, if the wave number is not chosen as an integer, the initial condition will not be a smooth periodic function in the given domain. Hence, the dissipative surface flux (`flux_lax_friedrichs` in this example) will introduce more dissipation. In particular, it will introduce more dissipation for "less smooth" initial data, corresponding to wave numbers `k` further away from integers.

We can compute the discrete derivative of the energy at the final time with respect to the wave number `k` as follows.

In [ ]:
ForwardDiff.derivative(energy_at_final_time, 1.0)

In [ ]:
dk_values = ForwardDiff.derivative.((energy_at_final_time,), k_values);
plot!(k_values, dk_values, label="Derivative")

### Postprocessing with Trixi2Vtk and ParaView

So far we have only visualized the solutions ad-hoc using the `plots(...)` function of the `Plots` package. However, for a more in-depth solution analysis especially of 3D data, it is often helpful to rely on a proper visualization program. Trixi supports converting its solution files, which are created as HDF5 files by the [SaveSolutionCallback](https://trixi-framework.github.io/Trixi.jl/stable/reference-trixi/#Trixi.SaveSolutionCallback)), to VTK files that can be opened and visualized with [ParaView](https://www.paraview.org) or [VisIt](http://visit.llnl.gov). This functionality is available in the [Trixi2Vtk](https://github.com/trixi-framework/Trixi2Vtk.jl) package, which provides a `trixi2vtk(...)` function:

In [ ]:
using Trixi2Vtk
trixi2vtk(joinpath("out", "solution_000000.h5"), output_directory="out")

This will create two files in the `out` directory, `solution_000000.vtu` and `solution_000000_celldata.vtu`. More information about how to use `trixi2vtk(...)` can be found in the [Trixi documentation](https://trixi-framework.github.io/Trixi.jl/stable/visualization/#Trixi2Vtk).

## Demo: Self-gravitating Sedov blast wave

Find details in [our paper](https://arxiv.org/abs/2008.10593)
and at the [reproducibility repository](https://github.com/trixi-framework/paper-self-gravitating-gas-dynamics).

<br/>

<div align="center">
  <img height="60%" src="https://github.com/trixi-framework/paper-self-gravitating-gas-dynamics/raw/master/assets/sedov-rho-phi-mesh.gif"
        alt="Simulation of self-gravitating Sedov blast wave with adaptive mesh refinement."/>
</div>

## Demo: MHD blast wave

In [ ]:
display_mp4("media/mhd_blast_wave.mp4")

## Demo: Kelvin-Helmholtz instability

In [ ]:
display_mp4("media/kelvin_helmholtz_instability.mp4")